In [64]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.212 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.5/166.8 GB disk)


In [65]:
!pip install wandb==0.15.0
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO

  Using cached wandb-0.15.0-py3-none-any.whl (2.0 MB)


Download Kaggle drone dataset UAV (https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav/data) and upload to notebook

In [66]:
!unzip drone_UAV.zip

Archive:  drone_UAV.zip
replace dataset_xml_format/dataset_xml_format/foto00088.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [67]:

if not(os.path.isdir('datasets')):
  !mkdir datasets
if not(os.path.isdir('datasets/train')):
  !mkdir datasets/train
if not(os.path.isdir('datasets/valid')):
  !mkdir datasets/valid

train_path='datasets/train/'
valid_path='datasets/valid/'

os.chdir("/content")
ano_paths=[]
for dirname, _, filenames in os.walk('drone_dataset_yolo/dataset_txt'):
    for filename in filenames:
        if filename[-4:]=='.txt':
            ano_paths+=[(os.path.join(dirname, filename))]

n=len(ano_paths)
print(n)
N=list(range(n))
random.shuffle(N)

train_ratio = 0.8
valid_ratio = 0.2


train_size = int(train_ratio*n)
valid_size = int(valid_ratio*n)

train_i = N[:train_size]
valid_i = N[train_size:train_size+valid_size]


print(train_i)
print(valid_i)

1360
[959, 1263, 408, 1055, 831, 459, 582, 426, 1053, 0, 341, 1185, 567, 901, 1313, 967, 583, 1163, 1221, 666, 1319, 1150, 1104, 734, 293, 1340, 1051, 945, 120, 404, 604, 78, 151, 228, 1001, 1060, 482, 1122, 282, 118, 428, 1265, 140, 1112, 863, 1031, 466, 778, 651, 1249, 952, 1108, 49, 224, 560, 513, 612, 969, 1348, 1270, 1226, 198, 992, 397, 964, 611, 475, 1149, 745, 832, 170, 758, 751, 869, 996, 441, 1346, 882, 821, 816, 485, 1093, 899, 1240, 1000, 544, 257, 99, 302, 393, 1014, 793, 1041, 411, 936, 442, 732, 388, 975, 486, 614, 234, 231, 1127, 1223, 334, 307, 830, 688, 432, 1186, 168, 154, 1191, 1, 1317, 128, 848, 362, 737, 539, 948, 505, 588, 115, 1258, 1291, 449, 438, 837, 1109, 714, 94, 164, 632, 731, 850, 423, 1217, 1228, 463, 685, 713, 535, 814, 883, 720, 1278, 97, 1210, 92, 943, 1337, 324, 1034, 925, 673, 760, 461, 190, 1161, 665, 1139, 1188, 1037, 274, 625, 434, 443, 71, 518, 161, 300, 618, 1096, 361, 366, 1039, 1141, 907, 1072, 175, 1233, 458, 1196, 246, 822, 481, 1329, 728, 

In [68]:
for i in train_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('drone_dataset_yolo/dataset_txt',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {train_path}
        !cp {img_path} {train_path}
    except:
        continue
print(len(os.listdir(train_path)))

cp: cannot stat 'drone_dataset_yolo/dataset_txt/classes.jpg': No such file or directory
2633


In [69]:
for i in valid_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('drone_dataset_yolo/dataset_txt',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {valid_path}
        !cp {img_path} {valid_path}
    except:
        continue
print(len(os.listdir(valid_path)))

1002


In [70]:
import yaml

data_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =1,
    names =['drone']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

%cat data.yaml

{names: [drone], nc: 1, test: test, train: train, val: valid}


In [71]:
names =['drone']
M=list(range(len(names)))
class_map=dict(zip(M,names))

In [72]:
model = YOLO("yolov8x.pt")

In [73]:
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=12 imgsz=480

Ultralytics YOLOv8.0.212 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=12, patience=50, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8

In [74]:
os.chdir("/content")
best_path = 'runs/detect/train/weights/best.pt'
model2 = YOLO(best_path)

In [75]:
!pip install pytube
from pytube import YouTube

if not(os.path.isdir('raw_videos')):
  !mkdir raw_videos


links = ["https://www.youtube.com/watch?v=DhmZ6W1UAv4", "https://www.youtube.com/watch?v=YrydHPwRelI"]

for l in links:
    vid = YouTube(l)
    vid.streams.filter(progressive="True").get_highest_resolution().download("/content/raw_videos")

In [76]:
os.chdir("/content")
vidpath = "raw_videos"

names = os.listdir(vidpath)
titles = []
for name in names:
	if(name.endswith('.mp4')):
		titles.append(name)

#make directories
if not(os.path.isdir('detections')):
  !mkdir detections
for title in titles:
  path = os.path.join('detections', title)
  if not(os.path.isdir(path)):
    os.mkdir(path)


In [77]:
titles

['Drone Tracking 1.mp4', 'Drone tracking 2.mp4']

In [78]:
os.listdir('/content/detections')

['Drone Tracking 1.mp4', 'Drone tracking 2.mp4']

In [79]:
os.chdir("/content")
for title in titles:
  video_path = os.path.join("/content/raw_videos", title)
  cap = cv2.VideoCapture(video_path)
  currentFrame = 0
  # Loop through the video frames
  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
          # Run YOLOv8 inference on the frame
          results = model2(frame, agnostic_nms=True)

          # Save result if detection found
          if len(results[0]) >= 1:
              annotated_frame = results[0].plot()
              image = np.array(annotated_frame)
              name = "/content/detections/{title}/frame_{frameNo}.jpg".format(title=title, frameNo=currentFrame)
              cv2.imwrite(name, image)

          currentFrame += 1

          # Break the loop if 'q' is pressed
          if cv2.waitKey(1) & 0xFF == ord("q"):
              break
      else:
          # Break the loop if the end of the video is reached
          os.chdir("/content")
          break

  # Release the video capture object and close the display window
  cap.release()



Streaming output truncated to the last 5000 lines.
0: 288x480 (no detections), 23.5ms
Speed: 1.8ms preprocess, 23.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.5ms
Speed: 1.3ms preprocess, 24.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 23.6ms
Speed: 1.3ms preprocess, 23.6ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.1ms
Speed: 1.7ms preprocess, 24.1ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 23.8ms
Speed: 1.8ms preprocess, 23.8ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 22.5ms
Speed: 1.4ms preprocess, 22.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.5ms
Speed: 1.9ms preprocess, 24.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no det

In [80]:
!pip install filterpy

In [81]:
from filterpy.kalman import KalmanFilter


In [82]:
#Configure Kalman Filter to initial start point
def initialize_filter(x, y):
  kf = KalmanFilter(dim_x=4, dim_z=2)
  kf.x = np.array([x, y, 0, 0])  # Initial state [x, y, dx, dy]
  kf.F = np.array([[1, 0, 1, 0],
                  [0, 1, 0, 1],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])  # State transition matrix
  kf.H = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0]])  # Measurement matrix
  kf.P *= 1e3  # Initial uncertainty
  kf.R = np.diag([1, 1])  # Measurement noise
  return kf


In [83]:
def update_kalman_filter(box):
    kf.predict()
    kf.update(box)
    return kf.x[:2]

In [84]:
for title in titles:
  #make directories
  if not(os.path.isdir('trajectories')):
    !mkdir trajectories
  for title in titles:
    path = os.path.join('trajectories', title)
    if not(os.path.isdir(path)):
      os.mkdir(path)

In [85]:
os.chdir("/content")
for title in titles:
  #make directories
  if not(os.path.isdir('final')):
    !mkdir final

In [86]:
img = cv2.imread('detections/Drone Tracking 1.mp4/frame_0.jpg')
print(img.shape)

(720, 1280, 3)


In [87]:
out_path = os.path.join("/content/final", title)
out_path

'/content/final/Drone tracking 2.mp4'

In [88]:
os.chdir("/content")
import re

#Create loop to go through each video input
for i, title in enumerate(titles):

  video_path = os.path.join("/content/raw_videos/", title)
  cap = cv2.VideoCapture(video_path)

  # Output video setup
  name = "/content/final/output_video_{i}.mp4".format(i=i)
  out_path = name
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

  # Create an empty canvas for drawing trajectories
  blank = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 3), dtype=np.uint8)
  currentFrame = 0
  isInitialized = False
  prev_position = None

  kf = None

  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
          # Run YOLOv8 inference on the frame
          results = model2(frame, agnostic_nms=True)

          # Only apply KF if detection in frame
          if len(results[0]) >= 1:

            # Initialize filter if not already and detection found
            if not isInitialized:

              box = results[0].boxes.xyxy.cpu()
              x = (int(box[0][0]) + int(box[0][2]))/2
              y = (int(box[0][1]) + int(box[0][3]))/2
              kf = initialize_filter(x, y)
              isInitialized = True
              b = np.array([x,y])
              estimated_position = update_kalman_filter(b)
              print("kalman filter initialized", isInitialized)


            # Else if filter exists, either predict or correct
            else:
              box = results[0].boxes.xyxy.cpu()
              x = (int(box[0][0]) + int(box[0][2]))/2
              y = (int(box[0][1]) + int(box[0][3]))/2
              b = np.array([x,y])
              estimated_position = update_kalman_filter(b)

            # Draw trajectory & detection box
            annotated_frame = results[0].plot()
            image = np.array(annotated_frame)
            cv2.circle(blank, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1)
            if currentFrame > 0 and prev_position is not None:
              cv2.line(blank, (int(prev_position[0]), int(prev_position[1])),
                      (int(estimated_position[0]), int(estimated_position[1])), (0, 255, 0), 2)

            prev_position = estimated_position.copy()

            #Overlay trajectory on frame
            overlay_frame = cv2.addWeighted(image, 0.8, blank, 0.2, 0)
            print("Trajectory drawn on image")

            #Save final frame

            video.write(overlay_frame)
            print("Frame written to video for frame: ", currentFrame)
            currentFrame += 1
          else:
            continue #Don't use frames without detections

      else:
          # Break the loop if the end of the video is reached
          os.chdir("/content")
          break

  # Release the video capture object and close the display window
  cap.release()
  video.release()
  cv2.destroyAllWindows()








Streaming output truncated to the last 5000 lines.
0: 288x480 (no detections), 23.8ms
Speed: 1.7ms preprocess, 23.8ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.5ms
Speed: 1.7ms preprocess, 24.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 23.0ms
Speed: 1.6ms preprocess, 23.0ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 25.3ms
Speed: 2.6ms preprocess, 25.3ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 23.4ms
Speed: 1.6ms preprocess, 23.4ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.2ms
Speed: 1.4ms preprocess, 24.2ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 23.6ms
Speed: 1.7ms preprocess, 23.6ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no det

Final output videos with trajectory and detections in 'final' folder